Jupyter Notebook that executes metric calculations for locally uploaded RDF files.

It imports functions from my_metrics.py and metrics_loader.py, computes selected metrics,

and writes the resulting metric table to a CSV file (consumed by the Streamlit app).

In [ ]:
# Parameters (will get overwritten)

# `endpoint_url`: URL of the SPARQL endpoint that will be queried for the ontology data.
# This is where the ontology is hosted and the metrics will be calculated based on its data.
# The value will be overwritten when the notebook runs with an actual SPARQL endpoint URL.
endpoint_url = "https://example.org/sparql"

# `default_graph`: The default graph used for the SPARQL query. This parameter is optional.
# If not provided, the query might use the default graph from the endpoint.
# It will be overwritten with a specific graph name if needed for the SPARQL query.
default_graph = "" # optional
# `metric_key`: The key that identifies which metric to calculate from the `DISPATCH` dictionary.
# The available options will be specified in the code, such as 'paths_depth', 'ont_tangledness', etc.
# This will be overwritten by the desired metric key for the analysis.
metric_key    = "paths_depth"
# `output_csv`: The name of the CSV file where the calculated metrics will be saved.
# This will be overwritten with a specific file path where the results will be stored.
output_csv    = "metrics.csv"

In [ ]:
import pandas as pd, scrapbook as sb
import my_metrics as mm

# Mapping: a dictionary that associates different metrics with their corresponding functions from `my_metrics` module.
DISPATCH = {
    "paths_depth":     mm.paths_depth_endpoint,
    "ont_tangledness":     mm.ont_tangledness_endpoint,
    "degree_variance": mm.degree_variance_endpoint,
    "primitives":      mm.primitives_endpoint,
    "depth_of_inheritance_tree": mm.depth_of_inh_tree_endpoint,
    "tbox":            mm.tbox_endpoint,
    "abox":            mm.abox_endpoint,
    "cohesion":        mm.cohesion_endpoint,
}

# Retrieving the corresponding calculation function based on the metric key (`metric_key`).
function = DISPATCH.get(metric_key)
# If the metric is not present in the DISPATCH dictionary, an error will be raised.
assert function, f"Unknown metric: {metric_key}"

# Calculating the metric by calling the corresponding function.
# The endpoint URL (`endpoint_url`) and an optional default graph (`default_graph`) are passed as parameters.
try:
    df = function(endpoint_url, default_graph or None)

except Exception as e:
    # Error handling: If an error occurs during metric calculation, en exception is raised
    raise e


# Saving the calculated results in the DataFrame `df` as a CSV file.
# `index=False` means that the DataFrame index will not be included in the CSV file.
df.to_csv(output_csv, index=False)

# Displaying calculated metrics in streamlit environment.
sb.glue("metrics_table", df)
